In [71]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community import retrievers 
from langchain.document_loaders import PyPDFLoader
import pprint
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [ ]:
llm=ChatOpenAI()
Embedding=OpenAIEmbeddings()
os.environ['OPEN_AI_API_KEY']=os.getenv('OPENAI_API_KEY')
loader=PyPDFLoader("THE_INDIAN_PENAL_CODE.pdf")
document=loader.load()
print(len(document))
pprint.pp(document[0].metadata)
full_text="\n".join([page.page_content for page in document])
merged_document=Document(page_content=full_text)
#print(full_text[:1000])
num_lines = full_text.count("\n") + 1  # Counting newlines
num_words = len(full_text.split())  # Splitting by whitespace to count words
num_chars = len(full_text)
print(len(document),num_lines,num_words,num_chars)










112
{'source': 'THE_INDIAN_PENAL_CODE.pdf', 'page': 0, 'page_label': '1'}
112 5947 78961 462897


In [80]:
text_spliiter=RecursiveCharacterTextSplitter
doc_chunk=text_spliiter(chunk_overlap=70,chunk_size=700).split_documents(document)
vectordb=FAISS.from_documents(doc_chunk,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
query="Extract all the INDIAN penal codes related information and punishment system"
retrived_doc=retriever.get_relevant_documents(query)

In [58]:
llm=ChatOpenAI(temperature=0)
prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based on the context:
    <context>
{context}
    </context>
    """
)

In [60]:
memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [87]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model_name="gpt-4", temperature=0),
    retriever=retriever,
    memory=memory,
    chain_type="stuff", # Explicitly set chain type
    verbose=True  
)

In [90]:
print(conversation_chain.run("Tell me the first 10 penal code in bullet format"))
print(conversation_chain.run("Tell me about harashment case scenario"))



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Tell me the most important penal code
Assistant: As an AI, I don't have opinions, but it's important to note that the significance of a penal code can vary depending on the context. However, the Indian Penal Code starts with the provision that every person shall be liable to punishment under this Code for every act or omission contrary to the provisions thereof, of which he shall be guilty within India. This could be seen as a fundamental principle of the code.
Human: Tell me the first 10 penal code
Assistant: The first 10 sections of the Indian Penal Code are as follows:

1. Title and extent of operation of the Code.
2. Punishment of offences committed within India.
3. Punishment of offences committed beyond, but which by law may be tried within, India.
4. Ext